# Data Preparation

In [ ]:
%%capture
from google.colab import drive
drive.mount('/content/gdrive')
!unzip "/content/gdrive/MyDrive/be521/final/pandora.zip" -d "/content"
!cd "/content"

In [ ]:
import scipy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from scipy.stats import pearsonr
from scipy import signal as sig

In [ ]:
X_train_raw = [np.load(f'X_train_{i}.npy', allow_pickle=True) for i in range(3)]
X_test_raw = [np.load(f'X_test_{i}.npy', allow_pickle=True) for i in range(3)]
Y_train_raw = [np.load(f'Y_train_{i}.npy', allow_pickle=True) for i in range(3)]
Y_test_raw = [np.load(f'Y_test_{i}.npy', allow_pickle=True) for i in range(3)]

In [ ]:
def NumWins(length, fs, winLen, winDisp):
    return round((length/fs-winLen+winDisp)/winDisp)

def get_features(filtered_window, fs=1000):
    return np.vstack([
        np.apply_along_axis(LL, 0, filtered_window),
        np.apply_along_axis(E, 0, filtered_window),
        np.apply_along_axis(BP, 0, filtered_window, f_min=75, f_max=115),
        np.apply_along_axis(BP, 0, filtered_window, f_min=125, f_max=159),
        np.apply_along_axis(BP, 0, filtered_window, f_min=159, f_max=175),
    ]).T

def get_windowed_feats(raw_ecog, fs, window_length, window_overlap):
    data = filter_data(raw_ecog)
    n = NumWins(len(data), fs, window_length, window_overlap)
    windows = []
    for i in range(n):
        start = round(i*window_overlap*fs)
        windows.append(get_features(data[start:start+round(window_length*fs),:]))
    return np.array(windows)

## Filter Data

In [ ]:
def filter_data(raw_eeg, fs=1000):
    sos = sig.butter(8, [0.15,200],btype='bandpass',output='sos',fs=fs)
    clean_data = sig.sosfiltfilt(sos, raw_eeg, axis=0)
    return clean_data

## Features

In [ ]:
# Line Length
def LL(x):
    return np.sum(np.abs(np.ediff1d(x)))

# Area
def A(x):
    return np.linalg.norm(x, 1)

# Energy
def E(x):
    return np.sum(np.square(x))

# Bandpower
def BP(x, f_min, f_max, fs=1000):
    f, Pxx = sig.periodogram(x, fs=fs)
    i_min = np.argmax(f>f_min)-1
    i_max = np.argmax(f>f_max)-1
    return np.trapz(Pxx[i_min:i_max], f[i_min:i_max])

## Execute

In [ ]:
# RUNTIME WARNING: 30 MIN
X_train = [get_windowed_feats(X_train_raw[i], 1000, 0.1, 0.05) for i in range(3)]
X_test = [get_windowed_feats(X_test_raw[i], 1000, 0.1, 0.05) for i in range(3)]

In [ ]:
for i in range(3):
    print(f"Subject {i+1}")
    print(f"X_train: {X_train[i].shape}")
    print(f"X_test:  {X_test[i].shape}")
    print("")

Subject 1
X_train: (9019, 62, 5)
X_test:  (2949, 62, 5)

Subject 2
X_train: (9019, 48, 5)
X_test:  (2949, 48, 5)

Subject 3
X_train: (9019, 64, 5)
X_test:  (2949, 64, 5)



In [ ]:
for i in range(3):
    X_train[i] = X_train[i].reshape(X_train[i].shape[0], -1)
    X_test[i] = X_test[i].reshape(X_test[i].shape[0], -1)

    print(f"Subject {i+1}")
    print(f"X_train: {X_train[i].shape}")
    print(f"X_test:  {X_test[i].shape}")
    print("")

Subject 1
X_train: (9019, 310)
X_test:  (2949, 310)

Subject 2
X_train: (9019, 240)
X_test:  (2949, 240)

Subject 3
X_train: (9019, 320)
X_test:  (2949, 320)



## Normalize X

In [ ]:
from sklearn.preprocessing import StandardScaler

# Subject 1
scaler1 = StandardScaler()
X_train[0] = scaler1.fit_transform(X_train[0])
X_test[0] = scaler1.transform(X_test[0])

# Subject 2
scaler2 = StandardScaler()
X_train[1] = scaler2.fit_transform(X_train[1])
X_test[1] = scaler2.transform(X_test[1])

# # Subject 3
scaler3 = StandardScaler()
X_train[2] = scaler3.fit_transform(X_train[2])
X_test[2] = scaler3.transform(X_test[2])

In [ ]:
from joblib import dump, load

dump(scaler1, 'scaler1.bin', compress=True)
dump(scaler2, 'scaler2.bin', compress=True)
dump(scaler3, 'scaler3.bin', compress=True)

['scaler3.bin']

## Downsample Y

In [ ]:
Y_train = [sig.resample(Y_train_raw[i], NumWins(len(Y_train_raw[i]), 1000, 0.1, 0.05)) for i in range(3)]
Y_test = [sig.resample(Y_test_raw[i], NumWins(len(Y_test_raw[i]), 1000, 0.1, 0.05)) for i in range(3)]

for i in range(3):
    print(f"Subject {i+1}")
    print(f"Y_train: {Y_train[i].shape}")
    print(f"Y_test:  {Y_test[i].shape}")
    print("")

Subject 1
Y_train: (9019, 5)
Y_test:  (2949, 5)

Subject 2
Y_train: (9019, 5)
Y_test:  (2949, 5)

Subject 3
Y_train: (9019, 5)
Y_test:  (2949, 5)



## Save & Load (to save runtime)

Save

In [ ]:
for i in range(3):
    np.save(f'processed_X_train_{i}.npy', X_train[i], allow_pickle=True)
    np.save(f'processed_X_test_{i}.npy', X_test[i], allow_pickle=True)
    np.save(f'processed_Y_train_{i}.npy', Y_train[i], allow_pickle=True)
    np.save(f'processed_Y_test_{i}.npy', Y_test[i], allow_pickle=True)

Load

In [ ]:
X_train = [np.load(f'processed_X_train_{i}.npy', allow_pickle=True) for i in range(3)]
X_test = [np.load(f'processed_X_test_{i}.npy', allow_pickle=True) for i in range(3)]
Y_train = [np.load(f'processed_Y_train_{i}.npy', allow_pickle=True) for i in range(3)]
Y_test = [np.load(f'processed_Y_test_{i}.npy', allow_pickle=True) for i in range(3)]

In [ ]:
for i in range(3):
    print(f"Subject {i+1}")
    print(f"X_train: {X_train[i].shape}")
    print(f"X_test:  {X_test[i].shape}")
    print(f"Y_train: {Y_train[i].shape}")
    print(f"Y_test:  {Y_test[i].shape}")
    print("")

Subject 1
X_train: (9019, 310)
X_test:  (2949, 310)
Y_train: (9019, 5)
Y_test:  (2949, 5)

Subject 2
X_train: (9019, 240)
X_test:  (2949, 240)
Y_train: (9019, 5)
Y_test:  (2949, 5)

Subject 3
X_train: (9019, 320)
X_test:  (2949, 320)
Y_train: (9019, 5)
Y_test:  (2949, 5)



Concat to one

In [ ]:
X = [np.vstack([X_train[i], X_test[i]]) for i in range(3)]
Y = [np.vstack([Y_train[i], Y_test[i]]) for i in range(3)]
Y_raw = [np.vstack([Y_train_raw[i], Y_test_raw[i]]) for i in range(3)]

## Train

In [ ]:
import lightgbm as lgb

lgb_train_data = [[lgb.Dataset(X[i], label=Y[i][:,j]) for j in range(5)] for i in range(3)]
lgb_val_data = [[lgb.Dataset(X_test[i], label=Y_test[i][:,j]) for j in range(5)] for i in range(3)]

params = {
    "learning_rate": 0.01,
    # "device_type": 'gpu',
    "max_leaves": 15,
    'max_depth': 4,
    "num_iterations": 500,
    "early_stopping_round": 10,
    "metric": 'rmse',
    "feature_fraction": 0.35,
    "bagging_fraction": 0.35,
    'bagging_freq': 1,
    'lambda_l1': 0.5,
    'lambda_l2': 0.5,
    # 'extra_trees': True,
    # 'min_data_in_leaf': 2,
    'max_bin': 25,
    'min_data_in_bin': 5,
    'random_state': 2
}

regs = []
for i in range(3):
    regs_subj = []
    for j in range(5):
        if j == 3:
            regs_subj.append(regs_subj[-1])
        else:
            reg = lgb.train(params, lgb_train_data[i][j], valid_sets=[lgb_val_data[i][j]])
            regs_subj.append(reg)
    regs.append(regs_subj)

/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Streaming output truncated to the last 5000 lines.
[46]	valid_0's rmse: 1.07867
[47]	valid_0's rmse: 1.07781
[48]	valid_0's rmse: 1.07721
[49]	valid_0's rmse: 1.07681
[50]	valid_0's rmse: 1.07657
[51]	valid_0's rmse: 1.07549
[52]	valid_0's rmse: 1.07484
[53]	valid_0's rmse: 1.07378
[54]	valid_0's rmse: 1.07345
[55]	valid_0's rmse: 1.07206
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[56]	valid_0's rmse: 1.07137
[57]	valid_0's rmse: 1.07109
[58]	valid_0's rmse: 1.07019
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[59]	valid_0's rmse: 1.06973
[60]	valid_0's rmse: 1.06951
[61]	valid_0's rmse: 1.06828
[62]	valid_0's rmse: 1.06756
[63]	valid_0's rmse: 1.06632
[64]	valid_0's rmse: 1.06604
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[65]	valid_0's rmse: 1.0655
[66]	valid_0's rmse: 1.06498
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[67]	valid_0's rmse: 1.0647
[68]	valid_0'

## Validation

In [ ]:
def avg_corr(pred, raw):
    return sum([pearsonr(pred[:,j], raw[:,j]).statistic for j in [0,1,2,4]])/4

from scipy.interpolate import CubicSpline

Y_train_pred = [None]*3
Y_val_pred = [None]*3
for i in range(3):
    Y_train_pred[i] = []
    Y_val_pred[i] = []
    for j in range(5):
        Y_train_pred[i].append(regs[i][j].predict(X[i], num_iteration=regs[i][j].best_iteration))
        Y_val_pred[i].append(regs[i][j].predict(X_test[i], num_iteration=regs[i][j].best_iteration))
    Y_train_pred[i] = np.hstack([vec.reshape(-1,1) for vec in Y_train_pred[i]])
    Y_val_pred[i] = np.hstack([vec.reshape(-1,1) for vec in Y_val_pred[i]])

Y_train_pred_intped = [
    np.array([CubicSpline(np.arange(len(Y_train_pred[i])), Y_train_pred[i][:,j], bc_type="natural")(np.linspace(0, len(Y_train_pred[i]), 598500)) for j in range(5)]).T for i in range(3)
]
Y_val_pred_intped = [
    np.array([CubicSpline(np.arange(len(Y_val_pred[i])), Y_val_pred[i][:,j], bc_type="natural")(np.linspace(0, len(Y_val_pred[i]), 147500)) for j in range(5)]).T for i in range(3)
]

for i in range(3):
    print("Subject", i+1)
    print("Train Corr: ", [pearsonr(Y_train_pred_intped[i][:,j], Y_raw[i][:,j]).statistic for j in range(5)])
    print("Val Corr: ", [pearsonr(Y_val_pred_intped[i][:,j], Y_test_raw[i][:,j]).statistic for j in range(5)])
    print("Average Train Corr: ", avg_corr(Y_train_pred_intped[i], Y_raw[i]))
    print("Average Val Corr: ", avg_corr(Y_val_pred_intped[i], Y_test_raw[i]))

Subject 1
Train Corr:  [0.5685715516473693, 0.6255566018537062, 0.5825189661071921, 0.1250747630937245, 0.569385641744906]
Val Corr:  [0.4927238665089544, 0.6728329092023869, 0.5726682251326455, 0.11177915241952628, 0.5390356694570458]
Average Train Corr:  0.5865081903382934
Average Val Corr:  0.5693151675752582
Subject 2
Train Corr:  [0.6500887826944182, 0.5731065082399903, 0.622863029860685, 0.16605528171418724, 0.5930799260623879]
Val Corr:  [0.6815223373234561, 0.6389991972777023, 0.6548042549633128, 0.20043087013898492, 0.6163846578510888]
Average Train Corr:  0.6097845617143703
Average Val Corr:  0.6479276118538899
Subject 3
Train Corr:  [0.690749512186343, 0.6243501725314453, 0.5952701112197447, 0.34089936326820536, 0.6508935037007537]
Val Corr:  [0.6555084420728332, 0.6211480063161687, 0.6163294390665547, 0.31290089019168577, 0.6921725756035533]
Average Train Corr:  0.6403158249095717
Average Val Corr:  0.6462896157647775


## Save

In [ ]:
def avg_corr(pred, raw):
    return sum([pearsonr(pred[:,j], raw[:,j]).statistic for j in [0,1,2,4]])/4

from scipy.interpolate import CubicSpline

Y_pred = [None]*3
for i in range(3):
    Y_pred[i] = []
    for j in range(5):
        Y_pred[i].append(regs[i][j].predict(X_test[i], num_iteration=regs[i][j].best_iteration))
    Y_pred[i] = np.hstack([vec.reshape(-1,1) for vec in Y_pred[i]])

Y_pred_intped = [
    np.array([CubicSpline(np.arange(len(Y_pred[i])), Y_pred[i][:,j], bc_type="natural")(np.linspace(0, len(Y_pred[i]), 147500)) for j in range(5)]).T for i in range(3)
]

for i in range(3):
    print("Subject", i+1)
    print("Val Corr: ", [pearsonr(Y_pred_intped[i][:,j], Y_test_raw[i][:,j]).statistic for j in range(5)])
    print("Average Val Corr: ", avg_corr(Y_pred_intped[i], Y_test_raw[i]))

Subject 1
Val Corr:  [0.4927238665089544, 0.6728329092023869, 0.5726682251326455, 0.11177915241952628, 0.5390356694570458]
Average Val Corr:  0.5693151675752582
Subject 2
Val Corr:  [0.6815223373234561, 0.6389991972777023, 0.6548042549633128, 0.20043087013898492, 0.6163846578510888]
Average Val Corr:  0.6479276118538899
Subject 3
Val Corr:  [0.6555084420728332, 0.6211480063161687, 0.6163294390665547, 0.31290089019168577, 0.6921725756035533]
Average Val Corr:  0.6462896157647775


In [ ]:
predictions = np.zeros((3,1), dtype=object)
predictions[0,0] = Y_val_pred_intped[0]
predictions[1,0] = Y_val_pred_intped[1]
predictions[2,0] = Y_val_pred_intped[2]

scipy.io.savemat("predictions.mat", {"predicted_dg": predictions})

In [ ]:
for i in range(3):
    for j in range(5):
        regs[i][j].save_model(f'lgb_subj{i}_finger{j}.txt')

## Test saved models

In [ ]:
import lightgbm as lgb
regs = []
for i in range(3):
    regs_subj = []
    for j in range(5):
        regs_subj.append(lgb.Booster(model_file=f'lgb_subj{i}_finger{j}.txt'))
    regs.append(regs_subj)

In [ ]:
def avg_corr(pred, raw):
    return sum([pearsonr(pred[:,j], raw[:,j]).statistic for j in [0,1,2,4]])/4

from scipy.interpolate import CubicSpline

Y_pred = [None]*3
for i in range(3):
    Y_pred[i] = []
    for j in range(5):
        Y_pred[i].append(regs[i][j].predict(X_test[i], num_iteration=regs[i][j].best_iteration))
    Y_pred[i] = np.hstack([vec.reshape(-1,1) for vec in Y_pred[i]])

Y_pred_intped = [
    np.array([CubicSpline(np.arange(len(Y_pred[i])), Y_pred[i][:,j], bc_type="natural")(np.linspace(0, len(Y_pred[i]), 147500)) for j in range(5)]).T for i in range(3)
]

for i in range(3):
    print("Subject", i+1)
    print("Val Corr: ", [pearsonr(Y_pred_intped[i][:,j], Y_test_raw[i][:,j]).statistic for j in range(5)])
    print("Average Val Corr: ", avg_corr(Y_pred_intped[i], Y_test_raw[i]))

Subject 1
Val Corr:  [0.4927238665089544, 0.6728329092023869, 0.5726682251326455, 0.11177915241952628, 0.5390356694570458]
Average Val Corr:  0.5693151675752582
Subject 2
Val Corr:  [0.6815223373234561, 0.6389991972777023, 0.6548042549633128, 0.20043087013898492, 0.6163846578510888]
Average Val Corr:  0.6479276118538899
Subject 3
Val Corr:  [0.6555084420728332, 0.6211480063161687, 0.6163294390665547, 0.31290089019168577, 0.6921725756035533]
Average Val Corr:  0.6462896157647775
